# From Square-Root Law to Dynamic Trade Effects

## 0. The dataset

In [ ]:
# Quote Data
# timestamp/trade index, bid, ask, bid volume, ask volume, midpoint

In [ ]:
# Trade Data
# timestamp/trade index, price, volume, trade type (buy +1/sell -1)

In [ ]:
# Derived Variables
# 

## 1. Baseline Implementation
- Square-Root Law: ΔP = Y σ√(Q/V)
- Parameter estimation and statistical validation
- Identify systematic deviations and failure modes

## 2. Traditional Model Extensions
- Propagator Model: P(t) = ∑G(t-s)ε(s)
- Temporal impact analysis (temporary vs permanent)
- Trade information content effects

## 3. ML Implementation
- Features: Volatility metrics, volume patterns, spreads, order book imbalance, decay patterns from propagator analysis
- Target: Direct price impact ΔP prediction
- Models: Compare ML predictions vs Square-root law vs Propagator model

## 4. Regime-Dependent Analysis
- Identify market conditions where traditional models underperform
- Cross-validation framework across different market regimes
- Performance comparison metrics focusing on model failure cases